In [1]:
import tensorflow as tf
import numpy as np

2023-07-10 03:58:43.177680: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class SpikingLayer(tf.keras.layers.Layer):
    def __init__(self, num_neurons, **kwargs):
        super(SpikingLayer, self).__init__(**kwargs)
        self.num_neurons = num_neurons

    def build(self, input_shape):
        self.kernel = self.add_weight("kernel", shape=[int(input_shape[-1]), self.num_neurons])
        self.threshold = self.add_weight("threshold", shape=[self.num_neurons])

    def call(self, inputs):
        # 计算输入与权重的点积
        outputs = tf.matmul(inputs, self.kernel)
        # 应用阈值函数
        outputs = tf.nn.relu(outputs - self.threshold)
        return outputs


In [3]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 数据预处理
train_images = train_images / 255.0
test_images = test_images / 255.0


In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),  # 将图像展平
    SpikingLayer(4),
    SpikingLayer(128),
    SpikingLayer(16), 
    tf.keras.layers.Dense(10) 
])


2023-07-10 03:58:45.135887: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images, 
    train_labels, 
    epochs=100,
    batch_size=10240,
    validation_data=(test_images, test_labels),
    verbose = 1
)


Epoch 1/100
6/6 [==============================] - 1s 51ms/step - loss: 2.2981 - accuracy: 0.1074 - val_loss: 2.2878 - val_accuracy: 0.1272
Epoch 2/100
6/6 [==============================] - 0s 18ms/step - loss: 2.2783 - accuracy: 0.1757 - val_loss: 2.2642 - val_accuracy: 0.2207
Epoch 3/100
6/6 [==============================] - 0s 18ms/step - loss: 2.2491 - accuracy: 0.2338 - val_loss: 2.2287 - val_accuracy: 0.2417
Epoch 4/100
6/6 [==============================] - 0s 15ms/step - loss: 2.2076 - accuracy: 0.2474 - val_loss: 2.1822 - val_accuracy: 0.2471
Epoch 5/100
6/6 [==============================] - 0s 16ms/step - loss: 2.1567 - accuracy: 0.2535 - val_loss: 2.1283 - val_accuracy: 0.2362
Epoch 6/100
6/6 [==============================] - 0s 16ms/step - loss: 2.1009 - accuracy: 0.2484 - val_loss: 2.0705 - val_accuracy: 0.2575
Epoch 7/100
6/6 [==============================] - 0s 17ms/step - loss: 2.0420 - accuracy: 0.2653 - val_loss: 2.0110 - val_accuracy: 0.2718
Epoch 8/100
6/6 [===

In [6]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)


313/313 [==============================] - 0s 712us/step - loss: 0.4286 - accuracy: 0.8710
Test accuracy: 0.8709999918937683


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 spiking_layer (SpikingLayer  (None, 4)                3140      
 )                                                               
                                                                 
 spiking_layer_1 (SpikingLay  (None, 128)              640       
 er)                                                             
                                                                 
 spiking_layer_2 (SpikingLay  (None, 16)               2064      
 er)                                                             
                                                                 
 dense (Dense)               (None, 10)                170       
                                                        

In [8]:
model.save("SNN_MNIST.h5")